# Absenteeism

* Data Source: Udemy - Data Science Bootcamp Program 2024
* Author: Huu Phuc (Felix) Hong
* Last Update: 9 January, 2024

## Library import

In [1]:
import pandas as pd

## Read data

In [2]:
raw_csv_data = pd.read_csv("Absenteeism_data.csv")

# Make a copy dataframe for manipulation purpose
df = raw_csv_data.copy()
df.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


There is no missing data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


## Preprocess data

#### Drop ID column

In [4]:
# Drop ID column
df.drop(['ID'], axis=1, inplace=True)

#### Get dummy variable

In [5]:
# Create dummy variables for Reason for Absence for checking
reason_columns = pd.get_dummies(df['Reason for Absence'])

# Ensure each person has only 1 reason for absenteeism (based on the context information)
reason_columns['check'] = reason_columns.sum(axis=1)
reason_columns['check'].value_counts()

1    700
Name: check, dtype: int64

In [6]:
# After confirming the validity of the constraint, drop check column
reason_columns = reason_columns.drop(['check'], axis = 1)

# Create dummy variables (drop first to avoid potential multicollinearity issues)
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first = True)

# Drop column after getting dummy variables
df.drop(['Reason for Absence'], axis=1, inplace=True)

#### Attributes Grouping

##### *Reason for absence*

Absences attested by the International Code of Diseases (ICD) classified into 21 categories as follows:
* 1 - Certain infectious and parasitic diseases
* 2 - Neoplasms
* 3 - Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism
* 4 - Endocrine, nutritional and metabolic diseases
* 5 - Mental and behavioural disorders
* 6 - Diseases of the nervous system
* 7 - Diseases of the eye and adnexa
* 8 - Diseases of the ear and mastoid process
* 9 - Diseases of the circulatory system
* 10 - Diseases of the respiratory system
* 11 - Diseases of the digestive system
* 12 - Diseases of the skin and subcutaneous tissue
* 13 - Diseases of the musculoskeletal system and connective tissue
* 14 - Diseases of the genitourinary system
* 15 - Pregnancy, childbirth and the puerperium
* 16 - Certain conditions originating in the perinatal period
* 17 - Congenital malformations, deformations and chromosomal abnormalities
* 18 - Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified
* 19 - Injury, poisoning and certain other consequences of external causes
* 20 - External causes of morbidity and mortality
* 21 - Factors influencing health status and contact with health services.

And 7 categories without (ICD) 

* 22 - Patient follow-up
* 23 - Medical consultation
* 24 - Blood donation 
* 25 - Laboratory examination
* 26 - Unjustified absence 
* 27 - Physiotherapy 
* 28 - Dental consultation

From the Reason Descriptions, we could further group these reasons into 4 big categories based on their similarities:
* Reasons 1-14 relate to various diseases.
* Reasons 15-17 relate to pregnancy.
* Reasons 18-21 relate to poisoning or uncategorized reasons.
* Reasons 22-28 relate to light reasons.

In [7]:
# Group reasons based on similarity
reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

In [8]:
# Concat to df
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)

##### *Education level*

We see that education column contains 4 values indicating education levels of the employees:
* 1 - High school
* 2 - Graduate
* 3 - Postgraduate
* 4 - Master or Doctor degree

However, because the number of employees reaching high school level accounts for majority of the data, we would group the other three levels into 1 category

In [9]:
df['Education'].value_counts()

1    583
3     73
2     40
4      4
Name: Education, dtype: int64

In [10]:
df['Education'] = df['Education'].map({1:0, 2:1, 3:1, 4:1})

#### Rename column

In [11]:
# Rename columns
df.rename(columns={0: "Reason_1", 1: "Reason_2", 2: "Reason_3", 3: "Reason_4"}, inplace=True)

#### Extract date information

In [12]:
# Convert Date column to datetime type
df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')

# Create month & day of the week columns
df['Month'] = df['Date'].dt.month
df['Day of the Week'] = df['Date'].apply(lambda x: x.weekday())

# Drop date column after extracting necessary information
df.drop(['Date'], axis=1, inplace=True)

#### Reorder columns

In [13]:
# Reorder columns
df = df[['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month', 'Day of the Week', 
        'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 
        'Body Mass Index', 'Education', 'Children', 'Pets', 'Absenteeism Time in Hours']]

In [14]:
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [16]:
# Save data
df.to_csv('Absenteeism_preprocessed.csv', index=False)